## ebs.ErrorBudget.run_etc Required Arguments
### Fixed:  
* output_dir  
* ref_json_filename  
* pp_json_filename  
* contrast_filename  
* target_list  
* luminosity  
* eeid  
* eepsr  
* npoints 

### Non-variable, mutable  
### Possibly variable  
* exo_zodi  
* wfe
* wfsc_factor
* sensitivity
* output_filename_prefix
* var_par
* *args  

In [1]:
import inspect
import yaml
import numpy as np
import ebs.ebs as ebs
import utilities.log_pdf as pdf

In [2]:
args_print = inspect.getfullargspec(ebs.ErrorBudget)[0]
print(
    f"ebs class arguments:\n{args_print}\n"
    )
args_print = inspect.getfullargspec(ebs.ErrorBudget.run_etc)[0]
print(
    f"ebs.run_etc method arguments:\n{args_print}"
      )

ebs class arguments:
['self', 'input_dir', 'output_dir', 'ref_json_filename', 'pp_json_filename', 'contrast_filename', 'target_list', 'luminosity', 'eeid', 'eepsr', 'exo_zodi', 'npoints']

ebs.run_etc method arguments:
['self', 'wfe', 'wfsc_factor', 'sensitivity', 'output_filename_prefix', 'var_par']


In [3]:
pdf.uniform

<function utilities.log_pdf.uniform(x, lower_bound, upper_bound)>

In [4]:
with open('mcmc.yml', 'r') as config:
    config = yaml.load(config, Loader=yaml.FullLoader)

In [5]:
# config

In [6]:
print(
    np.array(config['mcmc']['wfsc_factor']['prior_args']['lower_bound'])
)

[[9.9999e-07 9.9999e-07 9.9999e-07 9.9999e-07]
 [9.999e-05 9.999e-05 9.999e-05 9.999e-05]
 [0.00990099 0.00990099 0.00990099 0.00990099]
 [None None None None]
 [None None None None]]


In [7]:
# ErrorBudget class arguments
class_args = {
'input_dir': config['paths']['input'],
'output_dir': config['paths']['output'],
'ref_json_filename': config['json_files']['ref_json'],
'pp_json_filename': config['json_files']['pp_json'],
'contrast_filename': config['input_files']['contrast'],
'target_list': [config['targets'][star]['HIP'] 
               for star in config['targets']],
'luminosity': [config['targets'][star]['luminosity']
               for star in config['targets']],
'eeid': [config['targets'][star]['eeid'] 
               for star in config['targets']],
'eepsr': [config['targets'][star]['eepsr'] 
               for star in config['targets']],
'exo_zodi': [config['targets'][star]['exo_zodi'] 
               for star in config['targets']],
'npoints': 3
}

# ErrorBudget.run_etc method arguments
wfe = np.array(config['wfe'])
sensitivity = np.array(config['sensitivity'])
output_filename_prefix = config['output_filename_prefix']

# Undefined:  wfsc_factor, var_par, *args

In [8]:
error_budget = ebs.ErrorBudget(**class_args)

In [15]:
def ini_walker(config=config):
    walker_start = {}
    for key in config['mcmc'].keys():
        center = np.array(config['mcmc'][key]['ini_pars']['center'])
        spread = np.array(config['mcmc'][key]['ini_pars']['spread'])
                position = np.nan*np.ones(center.shape)
        index = np.where(center!=None)
        position[index] = (center[index] 
                + spread[index]*np.random.rand(*center.shape)[index])
        walker_start[key] = position
    return walker_start
        

        
    
    
#     [print(f"{key}\n{pars[key]}") for key in pars.keys()]
#     center = [np.array(pars[key][0][0]) for key in pars.keys()]
#     print("\nCENTER")
#     print(center)
# #     print(center.shape)
#     spread = [np.array(pars[key][0][1]) for key in pars.keys()]
#     print("\nSPREAD")
#     print(spread)
#     print(spread.shape)
#     ini_positn = []
#     for array in center:
#         positn = center + np.random.rand(array.shape)
#         ini_positn.append(1)
#     print(ini_positn)
    

IndentationError: unexpected indent (4206897078.py, line 6)

In [16]:
print(ini_walker())

[[9.9999e-07 9.9999e-07 9.9999e-07 9.9999e-07]
 [9.999e-05 9.999e-05 9.999e-05 9.999e-05]
 [0.00990099 0.00990099 0.00990099 0.00990099]
 [None None None None]
 [None None None None]]
[[9.9999e-09 9.9999e-09 9.9999e-09 9.9999e-09]
 [9.999e-07 9.999e-07 9.999e-07 9.999e-07]
 [9.90099e-05 9.90099e-05 9.90099e-05 9.90099e-05]
 [None None None None]
 [None None None None]]
[[1.00043332e-06 1.00766683e-06 1.00128433e-06 1.00327214e-06]
 [1.00528109e-04 1.00545574e-04 1.00892425e-04 1.00702280e-04]
 [9.92531894e-03 9.93703704e-03 9.91540212e-03 9.99837404e-03]
 [           nan            nan            nan            nan]
 [           nan            nan            nan            nan]]
[1.e-05]
[1.e-07]
[1.00823275e-05]
None
